In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

/home/nlp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
dbpedia = tf.contrib.learn.datasets.load_dataset(
      'dbpedia', test_with_fake_data=False, size='small')

Instructions for updating:
Please use tf.data.
Instructions for updating:
See contrib/learn/README.md
Instructions for updating:
See contrib/learn/README.md
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded dbpedia_csv.tar.gz 68431223 bytes.
Instructions for updating:
Use tf.data instead.
Instructions for updating:
Use tf.data instead.


In [3]:
x_train = pd.DataFrame(dbpedia.train.data)[1]
y_train = pd.Series(dbpedia.train.target)
x_test = pd.DataFrame(dbpedia.test.data)[1]
y_test = pd.Series(dbpedia.test.target)

In [4]:
print(x_train[0])
print(len(x_train[0]))

 Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.
248


In [5]:
y_train[0]

1

In [6]:
x_train[0]

' Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.'

In [18]:
# Process vocabulary
MAX_DOCUMENT_LENGTH = 100
char_processor = tf.contrib.learn.preprocessing.ByteProcessor(
  MAX_DOCUMENT_LENGTH)
x_train = np.array(list(char_processor.fit_transform(x_train)))
x_test = np.array(list(char_processor.transform(x_test)))

In [19]:
x_train[0]

array([ 32,  65,  98,  98, 111, 116, 116,  32, 111, 102,  32,  70,  97,
       114, 110, 104,  97, 109,  32,  69,  32,  68,  32,  65,  98,  98,
       111, 116, 116,  32,  76, 105, 109, 105, 116, 101, 100,  32, 119,
        97, 115,  32,  97,  32,  66, 114, 105, 116, 105, 115, 104,  32,
        99, 111,  97,  99, 104,  98, 117, 105, 108, 100, 105, 110, 103,
        32,  98, 117, 115, 105, 110, 101, 115, 115,  32,  98,  97, 115,
       101, 100,  32, 105, 110,  32,  70,  97, 114, 110, 104,  97, 109,
        32,  83, 117, 114, 114, 101, 121,  32, 116], dtype=uint8)

In [7]:
vocab_file = 'data/vocab-2016-09-10.txt'

In [10]:
from char_data import Batcher
# Create a Batcher to map text to character ids.
vocab_file = 'data/vocab-2016-09-10.txt'
batcher = Batcher(vocab_file, 50)

In [11]:
# Input placeholders to the biLM.
context_character_ids = tf.placeholder('int32', shape=(None, None, 50))
question_character_ids = tf.placeholder('int32', shape=(None, None, 50))

In [12]:
# Now we can compute embeddings.
raw_context = [
    'Pretrained biLMs compute representations useful for NLP tasks .',
    'They give state of the art performance for many tasks .'
]
tokenized_context = [sentence.split() for sentence in raw_context]

In [13]:
context_ids = batcher.batch_sentences(tokenized_context)

In [15]:
context_ids.shape

(2, 13, 50)

In [1]:
class config:
    vocab_size = 2000
    embedding_size = 100
    filter_size = 64
    num_layers = 3
    block_size = 3
    filter_h = 5
    context_size = 20
    text_size = context_size
    batch_size = 32
    epochs = 5
    num_sampled = 64
    learning_rate = 0.0001
    momentum = 0.99
    grad_clip = 0.1
    num_batches = 0
    ckpt_path = 'ckpt'
    summary_path = 'logs'
    #data_dir = "data/texts/reviews/movie_reviews"
    data_dir = "data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled"

In [2]:
from model import GatedCharCNN
from conf_utils import *

/home/nlp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#Initialize configuration files
conf = prepare_conf(config)
gcnn = GatedCharCNN(conf)

Please download the data as mentioned in Requirements
Tensor("concat:0", shape=(?, ?, 124), dtype=float32)
Tensor("Reshape_1:0", shape=(?, ?, 100), dtype=float32)


In [1]:
from bilm.training import train, load_options_latest_checkpoint, load_vocab
from bilm.data import BidirectionalLMDataset

/home/nlp/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
vocab_file = 'data/vocab-2016-09-10.txt'
vocab = load_vocab(vocab_file, 50)

In [ ]:
vocab.size()

In [3]:
# define the options
batch_size = 128  # batch size for each GPU
n_gpus = 3

# number of tokens in training data (this for 1B Word Benchmark)
n_train_tokens = 768648884

options = {
     'bidirectional': True,

     'char_cnn': {'activation': 'relu',
      'embedding': {'dim': 16},
      'filters': [[1, 32],
       [2, 32],
       [3, 64],
       [4, 128],
       [5, 256],
       [6, 512],
       [7, 1024]],
      'max_characters_per_token': 50,
      'n_characters': 261,
      'n_highway': 2},
    
     'dropout': 0.1,
    
     'lstm': {
      'cell_clip': 3,
      'dim': 4096,
      'n_layers': 2,
      'proj_clip': 3,
      'projection_dim': 512,
      'use_skip_connections': True},
    
     'all_clip_norm_val': 10.0,
    
     'n_epochs': 10,
     'n_train_tokens': n_train_tokens,
     'batch_size': batch_size,
     'n_tokens_vocab': vocab.size,
     'unroll_steps': 20,
     'n_negative_samples_batch': 8192,
    }

In [6]:
train_prefix = 'data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/*'

In [7]:
data = BidirectionalLMDataset(train_prefix, vocab, test=False,
                                      shuffle_on_load=True)

Found 99 shards at data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/*
Loading data from: data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/news.en-00034-of-00100
Loaded 305408 sentences.
Finished loading
Found 99 shards at data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/*
Loading data from: data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/news.en-00080-of-00100
Loaded 305615 sentences.
Finished loading


In [13]:
data_gen = data.iter_batches(batch_size * 1, 20)

In [14]:
for batch_no, batch in enumerate(data_gen, start=1):

            # slice the input in the batch for the feed_dict
            X = batch
            break

Loading data from: data/1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/news.en-00027-of-00100
Loaded 306804 sentences.
Finished loading


In [17]:
X['tokens_characters'].shape

(128, 20, 50)

In [16]:
X.keys()

dict_keys(['token_ids', 'tokens_characters', 'next_token_id', 'token_ids_reverse', 'tokens_characters_reverse', 'next_token_id_reverse'])